#                    ARUMA 
$\textit{"Bajo el cielo estrellado navego en el mundo"}$

In [115]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as kt
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from bioinfokit.visuz import cluster
from datetime import date 
from sklearn import linear_model
from scipy.stats import skew
import statsmodels.api as sm
import requests
from plotnine import ggplot, aes, geom_point
from forex_python.converter import CurrencyRates
from bs4 import BeautifulSoup
import datetime
from haversine import haversine, Unit
import math
from geopy.distance import geodesic
import openpyxl

### Alternativa manual

In [10]:
def distancia_entre_puntos(lat1, lon1, lat2, lon2):
    
    #Radio de la Tierra en metros
    
    Earth_radius = 6371000
    
    lat1, lon1, lat2, lon2, = map(math.radians, [lat1,lon1,lat2,lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    #Formula de haversine para calcular la distancia.
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * sin(dlon / 2) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    distance = Earth_radius * c
    
    return distance

## Usando Geopy

In [101]:
#Cargamos el dataset original
dataset_original = pd.read_excel("Datasets/dataset_tasacion_train.xlsx", header=[0], index_col=None)

#Cargamos todos los datasets complementarios LIMA

df_bancos = pd.read_excel("Datasets/Datasets_complement/bancos_lima.xlsx", header=None, index_col=None)
df_bares = pd.read_excel("Datasets/Datasets_complement/bares_lima.xlsx", header=[0], index_col=None)
df_centros_comerciales = pd.read_excel("Datasets/Datasets_complement/centros_comerciales_lima.xlsx", header=[0], index_col=None)
df_hospitales = pd.read_excel("Datasets/Datasets_complement/hospitales_lima.xlsx", header=[0], index_col=None)
df_Estaciones_publicas = pd.read_excel("Datasets/Datasets_complement/Metro_autobus_lima.xlsx", header=[0], index_col=None)
df_parques = pd.read_excel("Datasets/Datasets_complement/parques_lima.xlsx", header=[0], index_col=None)
df_playas = pd.read_excel("Datasets/Datasets_complement/playas_lima.xlsx", header=[0], index_col=None)
df_restaurants = pd.read_excel("Datasets/Datasets_complement/restaurantes_lima.xlsx", header=[0], index_col=None)
df_museos = pd.read_excel("Datasets/Datasets_complement/museos_lima.xlsx", header=[0], index_col=None)


#Fixers
df_bancos.rename(columns={3:'Latitud (Decimal)'}, inplace= True)
df_bancos.rename(columns={4:'Longitud (Decimal)'}, inplace= True)



In [105]:
#df_hospitales.head()
#df_bares.head()
df_museos.head()

,Text,Text1,Latitud (Decimal),Longitud (Decimal)
0,C. Pl. Bolívar,"Pueblo Libre 15084, Perú",-12.077900,-77.064855
1,Jirón de la Unión 1030,"Jirón de la Unión 1030, Lima 15001, Perú",-12.052048,-77.038171
2,Av. Simón Bolivar 1515,"Av. Simón Bolivar 1515, Pueblo Libre 15084, Perú",-12.072233,-77.072823
3,Plaza Bolívar,"Av. Abancay, Lima 15001, Perú",-12.047623,-77.028234
4,Jr. Áncash 213,"Jr. Áncash 213, Lima 15001, Perú",-12.044831,-77.030866


In [106]:
#FIXERS DE COLUMNAS:

df_bares.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_bares.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_centros_comerciales.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_centros_comerciales.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_hospitales.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_hospitales.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_Estaciones_publicas.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_Estaciones_publicas.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_parques.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_parques.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_playas.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_playas.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_restaurants.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_restaurants.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)

df_museos.rename(columns={'latitud':'Latitud (Decimal)'}, inplace= True)
df_museos.rename(columns={'longitud':'Longitud (Decimal)'}, inplace= True)


In [96]:
#df_bancos.head()
#df_bares.head()
#df_centros_comerciales.head()
#df_hospitales.head()
#df_Estaciones_publicas.head()
#df_parques.head()
#df_playas.head()
#df_restaurants.head()

#dataset_original.info()

In [107]:
def contador_cercanos_validados(row, df):
    """
    Calcula la cantidad de puntos en un DataFrame que están a una distancia menor o igual a 1.5 km de un punto dado
    en una fila de otro DataFrame. Se verifica que las coordenadas estén dentro del rango válido.
    """
    latitud = row['Latitud (Decimal)']
    longitud = row['Longitud (Decimal)']
    
    # Chequeo de valores válidos para latitud y longitud
    if not (-90 <= latitud <= 90) or not (-180 <= longitud <= 180):
        return 0
    
    # Crear una serie de booleanos que indica si cada punto está dentro del radio
    dentro_radio = df.apply(lambda p: geodesic((latitud, longitud), (p['Latitud (Decimal)'], p['Longitud (Decimal)'])).km <= 1.5, axis=1)
    # Contar cuántos puntos están dentro del radio
    return dentro_radio.sum()


## Vamos a calcular el conteo de puntos cercanos para cada punto del dataset original

In [108]:
dataset_original['#de_bancos'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_bancos), axis=1)
dataset_original['#de_bares'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_bares), axis=1)
dataset_original['#de_centros_comerciales'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_centros_comerciales), axis=1)
dataset_original['#de_hospitales'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_hospitales), axis=1)
dataset_original['#de_Estaciones_publicas'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_Estaciones_publicas), axis=1)
dataset_original['#de_parques'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_parques), axis=1)
dataset_original['#de_playas'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_playas), axis=1)
dataset_original['#de_restaurantes'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_restaurants), axis=1)
dataset_original['#de_museos'] = dataset_original.apply(lambda row: contador_cercanos_validados(row, df_museos), axis=1)

In [109]:
dataset_original[:50]

,Fecha entrega del Informe,Calle,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),...,Valor comercial,#de_bancos,#de_bares,#de_centros_comerciales,#de_hospitales,#de_Estaciones_publicas,#de_parques,#de_playas,#de_restaurantes,#de_museos
0,2016-09-01 19:00:00,REBECA OQUENDO,NaN,Lima,Lima,Breña,1.0,0.0,-12.059545,-77.046595,...,83754,43,63,14,2,1,1,7,6,10
1,2021-03-11 19:00:00,CONDOMINIO DOSS,NaN,Lima,Lima,Comas,0.0,0.0,-11.937662,-77.066525,...,85355,0,0,0,0,0,0,0,0,0
2,2019-03-18 19:00:00,MANZANA 14 LT 10 LOS JARDINES DEL GOLF,NaN,La Libertad,Trujillo,Victor Larco Herrera,0.0,0.0,-8.135306,-79.035518,...,83838,0,0,0,0,0,0,0,0,0
3,2019-07-25 19:00:00,TRINIDAD MORAN,NaN,Lima,Lima,Lince,NaN,NaN,-12.089796,-77.039245,...,111619,4,5,5,2,2,2,0,17,5
4,2018-01-01 19:00:00,MZ 8 AC,NaN,Lima,Lima,Villa María del Triunfo,1.0,0.0,-12.181048,-76.927642,...,305191,0,0,0,0,0,0,0,2,0
5,2021-01-27 19:00:00,ABRAHAM VALDELOMAR,NaN,Lima,Lima,Pueblo Libre,0.0,0.0,-12.073305,-77.067643,...,80264,0,1,0,0,0,0,0,3,3
6,2016-03-07 19:00:00,31,NaN,Lima,Lima,Lima,1.0,1.0,-12.107400,-77.014200,...,290088,1,0,0,0,0,0,0,2,0
7,2021-01-06 19:00:00,MANZANOS,NaN,Lima,Lima,San Juan de Lurigancho,0.0,0.0,-11.959087,-76.992018,...,136930,0,0,0,0,0,0,0,0,0
8,2018-05-03 19:00:00,RIO,NaN,Lima,Lima,Pueblo Libre,1.0,0.0,-12.075226,-77.056260,...,141126,5,2,4,0,1,0,0,3,3
9,2016-06-19 19:00:00,S/N,NaN,Lima,Lima,Comas,0.0,0.0,-11.931685,-77.059962,...,60405,0,0,0,0,0,0,0,0,0


In [118]:
dataset_Lima = dataset_original.copy()

writer = pd.ExcelWriter('Datasets/Dataset_Lima_train.xlsx', engine='openpyxl')
dataset_Lima.to_excel(writer, sheet_name='Hoja1', index=False)
writer.save()

/tmp/ipykernel_7599/551980194.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
